In [24]:
%cd /home/xuwei/training
!wget -c https://alphacephei.com/vosk-colab/kaldi.tar.gz
!tar xzf kaldi.tar.gz

/home/xuwei/training
--2025-04-11 17:00:55--  https://alphacephei.com/vosk-colab/kaldi.tar.gz
正在解析主机 alphacephei.com (alphacephei.com)... 2a01:4f8:13a:279f::2, 188.40.21.16
正在连接 alphacephei.com (alphacephei.com)|2a01:4f8:13a:279f::2|:443... 已连接。


/home/xuwei/vosk_build2/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


已发出 HTTP 请求，正在等待回应... 200 OK
长度： 1041599901 (993M) [application/octet-stream]
正在保存至: ‘kaldi.tar.gz’

kaldi.tar.gz        100%[===================>] 993.35M  13.1MB/s    用时 76s     

2025-04-11 17:02:11 (13.0 MB/s) - 已保存 ‘kaldi.tar.gz’ [1041599901/1041599901])



In [36]:
!export KALDI_ROOT=/home/xuwei/training/kaldi

!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/xuwei/training/kaldi/tools/openfst-1.8.0/lib/fst
!source /home/xuwei/training/vosk-api/training/cmd.sh
!source /home/xuwei/training/vosk-api/training/path.sh

The standard file /tools/config/common_path.sh is not present -> Exit!


In [25]:
%cd /home/xuwei/training
!rm -rf vosk-api
!git clone https://github.com/alphacep/vosk-api

/home/xuwei/training
正克隆到 'vosk-api'...
remote: Enumerating objects: 4380, done.
remote: Counting objects: 100% (700/700), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 4380 (delta 592), reused 450 (delta 450), pack-reused 3680 (from 2)
接收对象中: 100% (4380/4380), 14.15 MiB | 12.46 MiB/s, 完成.
处理 delta 中: 100% (2421/2421), 完成.


In [37]:
!rm -rf /home/xuwei/training/kaldi/egs/ac/training
!cp -r /home/xuwei/training/vosk-api/training /home/xuwei/training/kaldi/egs/ac

shell-init: 获取当前目录时出错：getcwd: 无法访问父目录：没有那个文件或目录
shell-init: 获取当前目录时出错：getcwd: 无法访问父目录：没有那个文件或目录


In [4]:
!apt install flac

E: 无法打开锁文件 /var/lib/dpkg/lock-frontend - open (13: 权限不够)
E: 无法获取 dpkg 前端锁 (/var/lib/dpkg/lock-frontend)，请查看您是否正以 root 用户运行？


In [34]:

%cd /home/xuwei/training/kaldi/egs/ac/training
!rm -rf exp
!sed -i 's:--nj 10:--nj 2:g' run.sh
!cat run.sh
!bash run.sh --stop-stage 3

/home/xuwei/training/kaldi/egs/ac/training
#!/usr/bin/env bash

. ./cmd.sh
. ./path.sh

stage=-1
stop_stage=100

. utils/parse_options.sh

# Data preparation
if [ ${stage} -le 0 ] && [ ${stop_stage} -ge 0 ]; then
  data_url=www.openslr.org/resources/31
  lm_url=www.openslr.org/resources/11
  database=corpus

  mkdir -p $database
  for part in dev-clean-2 train-clean-5; do
    local/download_and_untar.sh $database $data_url $part
  done

  local/download_lm.sh $lm_url $database data/local/lm

  local/data_prep.sh $database/LibriSpeech/train-clean-5 data/train
  local/data_prep.sh $database/LibriSpeech/dev-clean-2 data/test
fi

# Dictionary formatting
if [ ${stage} -le 1 ] && [ ${stop_stage} -ge 1 ]; then
  local/prepare_dict.sh data/local/lm data/local/dict
  utils/prepare_lang.sh data/local/dict "<UNK>" data/local/lang data/lang
fi

# Extract MFCC features
if [ ${stage} -le 2 ] && [ ${stop_stage} -ge 2 ]; then
  for task in train; do
    steps/make_mfcc.sh --cmd "$train_cmd" --nj 2 dat

In [35]:
!sed -i 's:--nj 10:--nj 2:g' local/chain/run_tdnn.sh
!sed -i 's:steps/nnet3/chain/train.py:steps/nnet3/chain/train.py --use-gpu false:g' local/chain/run_tdnn.sh
!cat local/chain/run_tdnn.sh
!bash local/chain/run_tdnn.sh

#!/bin/bash

# Set -e here so that we catch if any executable fails immediately
set -euo pipefail

# (some of which are also used in this script directly).
stage=-1
decode_nj=10
train_set=train
gmm=tri3
nnet3_affix=
suffix=

# The rest are configs specific to this script.  Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=   # affix for the TDNN directory name
tree_affix=
train_stage=-10
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
chunk_width=140,100,160
common_egs_dir=
xent_regularize=0.1
dropout_schedule='0,0@0.20,0.5@0.50,0'

# training options
srand=0
remove_egs=true

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

# Problem: We have removed the "train_" prefix of our training set in
# the alignment directory names! Bad!
gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali
tree_dir=exp/chain${suffix}/tree${tree_affix:+_$tree_affix}
lang=data/l

In [32]:
!cp exp/chain/tdnn/13.mdl exp/chain/tdnn/final.mdl

cp: 对 'exp/chain/tdnn/13.mdl' 调用 stat 失败: 没有那个文件或目录


In [18]:
!bash run.sh --stage 5

Converting 'data/local/lm/lm_tgsmall.arpa.gz' to FST
cp: cannot stat 'data/lang/topo': No such file or directory
mkgraph.sh: expected data/lang_test/L_disambig.fst to exist
cat: data/lang/oov.int: No such file or directory
utils/build_const_arpa_lm.sh: can't find oov symbol id in data/lang/oov.int
steps/make_mfcc.sh --cmd run.pl --nj 2 data/test exp/make_mfcc/test
utils/validate_data_dir.sh: Successfully validated data-directory data/test
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
run.pl: 2 / 2 failed, log is in exp/make_mfcc/test/make_mfcc_test.*.log
steps/compute_cmvn_stats.sh data/test exp/make_mfcc/test
steps/compute_cmvn_stats.sh: no such file data/test/feats.scp
steps/online/nnet2/extract_ivectors_online.sh --nj 2 data/test exp/chain/extractor exp/chain/ivectors_test
steps/online/nnet2/extract_ivectors_online.sh: No such file data/test/feats.scp
steps/nnet3/decode.sh --cmd run.pl --num-threads 10 --nj 1 --beam 13.0 --max-active 700